# Airbnb House Analysis and Recommender Application

##### Jie Bao, Kuangyi Zhang, Lanny Xu,
##### Dr. Bamshad Mobasher, Spring 2018

## Import Library

In [3]:
%pylab inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn import *
from sklearn import neighbors, tree, naive_bayes, cross_validation
from sklearn.cross_validation import KFold
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.linear_model import LinearRegression, Lasso, Ridge, ElasticNet, SGDRegressor
from sklearn import preprocessing
import operator

Populating the interactive namespace from numpy and matplotlib


## Read in data and preprocessing

In [11]:
listings = pd.read_table("listings_edited.csv", header='infer', delimiter=",")
print listings.shape
listings.head(5)

(5207, 39)


,id,listing_url,name,summary,picture_url,host_response_time,host_response_rate,host_is_superhost,host_identity_verified,neighbourhood_cleansed,...,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,instant_bookable,cancellation_policy,reviews_per_month
0,13824783,https://www.airbnb.com/rooms/13824783,Full floor of a city cottage (up to 4 guests),This 3rd-floor garrett apartment (built-out at...,https://a0.muscache.com/im/pictures/510d45f8-e...,within an hour,100%,t,t,Lincoln Square,...,99.0,10.0,10.0,10.0,10.0,10.0,10.0,t,moderate,2.57
1,16740225,https://www.airbnb.com/rooms/16740225,Guest Room of Two Bedroom Condo,Warm and secure room in a Two Bedroom Condo lo...,https://a0.muscache.com/im/pictures/697182d3-d...,within an hour,100%,f,t,Lincoln Square,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,t,strict,NaN
2,18125245,https://www.airbnb.com/rooms/18125245,"Cozy, spacious 2 flat in Lincoln Square!","This cute, spacious 2 flat in Lincoln Square i...",https://a0.muscache.com/im/pictures/bf761217-c...,within an hour,100%,f,t,Lincoln Square,...,95.0,10.0,10.0,10.0,10.0,10.0,10.0,f,moderate,4.00
3,8362570,https://www.airbnb.com/rooms/8362570,Lincoln Square Ravenswood,"Lincoln square, Ravenswood and Andersonville g...",https://a0.muscache.com/im/pictures/224a38c0-a...,within a few hours,100%,t,t,Lincoln Square,...,98.0,10.0,10.0,10.0,10.0,10.0,10.0,f,flexible,0.87
4,789867,https://www.airbnb.com/rooms/789867,Cozy Private Room in a Classic Chicago Appartm...,$40 OFF COUPON FOR ALL NEW AIRBNB GUESTS - VIS...,https://a0.muscache.com/im/pictures/72031963/0...,within an hour,100%,f,t,Lincoln Square,...,94.0,9.0,9.0,10.0,10.0,10.0,9.0,t,strict,1.96


In [13]:
listings.describe(include="all").T

,count,unique,top,freq,mean,std,min,25%,50%,75%,max
id,5207,NaN,NaN,NaN,1.1491e+07,5.32038e+06,2384,7.24207e+06,1.30835e+07,1.57733e+07,1.86568e+07
listing_url,5207,5207,https://www.airbnb.com/rooms/6558625,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
name,5206,5169,Wells St Suites - Old Town 3 Bed,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN
summary,5125,4841,This property offers luxury in Chicago’s trend...,23,NaN,NaN,NaN,NaN,NaN,NaN,NaN
picture_url,5207,5202,https://a0.muscache.com/im/pictures/70087089/b...,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN
host_response_time,4951,4,within an hour,3257,NaN,NaN,NaN,NaN,NaN,NaN,NaN
host_response_rate,4951,55,100%,3873,NaN,NaN,NaN,NaN,NaN,NaN,NaN
host_is_superhost,5207,2,f,3840,NaN,NaN,NaN,NaN,NaN,NaN,NaN
host_identity_verified,5207,2,t,4093,NaN,NaN,NaN,NaN,NaN,NaN,NaN
neighbourhood_cleansed,5207,72,West Town,729,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [14]:
listings.dtypes

id                               int64
listing_url                     object
name                            object
summary                         object
picture_url                     object
host_response_time              object
host_response_rate              object
host_is_superhost               object
host_identity_verified          object
neighbourhood_cleansed          object
state                           object
city                            object
zipcode                         object
property_type                   object
room_type                       object
accommodates                     int64
bathrooms                      float64
bedrooms                       float64
beds                           float64
bed_type                        object
amenities                       object
price                           object
security_deposit                object
cleaning_fee                    object
guests_included                  int64
extra_people             

In [15]:
# Id, Listing_url,Name, Summary, Picture_url
# Host_response_time, Host_response_rate, Host_identity_verified, Neighbourhood_cleansed
# City, State, Zipcode
# Property_type, Room_type, Accommodates, Bathrooms, bedrooms, Beds, Bed_type,
# Amenities: TV, wireless internet, air condition, heating, pets, washer, dryer
# Price, Security_deposit, Cleaning_fee, Guests_included, extra_people, minimum_nights, maximum_nights
# Number_of_reviews, Review_scores_rating, Instant_bookable, Cancellation_policy, reviews_per_month